# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import gmaps.datasets

# Import API key
from api_keys2 import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
output_data = "cities.csv"
df = pd.read_csv(output_data)
df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mareeba,-17.0000,145.4333,87.800,66,75,3.09,AU,1610243809
1,Albany,42.6001,-73.9662,26.996,59,100,3.44,US,1610243707
2,Atuona,-9.8000,-139.0333,78.260,77,69,7.02,PF,1610243809
3,Hermanus,-34.4187,19.2345,62.996,93,100,5.55,ZA,1610243280
4,Saskylakh,71.9167,114.0833,-26.176,84,100,4.11,RU,1610243809
...,...,...,...,...,...,...,...,...,...
557,Marakkanam,12.2000,79.9500,76.838,87,44,4.52,IN,1610243881
558,Umeå,63.8284,20.2597,30.200,69,90,6.17,SE,1610243881
559,Luau,-10.7073,22.2247,62.312,99,82,1.45,AO,1610243881
560,Agadez,19.7500,10.2500,65.354,26,0,2.11,NE,1610243881


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [15]:
#Set locations and weight
humidity = df['Humidity'].astype(float)

locations = df[['Lat', 'Lng']]

fig=gmaps.figure()

#Add heatmap layer
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=5,
                                point_radius = 2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [30]:
ideal_df = df.loc[(df['Max Temp'] < 30) & 
                  (df['Max Temp'] > 24) & 
                  (df['Wind Speed'] < 8) &
                 (df['Cloudiness'] == 0)]
ideal_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
168,Vestmannaeyjar,63.4427,-20.2734,27.446,88,0,0.90,IS,1610243830
306,Hofn,64.2539,-15.2082,24.368,82,0,3.91,IS,1610243847
414,Taku,39.0211,117.6469,26.600,31,0,6.00,CN,1610243861
428,Ziarat,30.3823,67.7282,24.458,38,0,0.70,PK,1610243862
539,Bǝrdǝ,40.3744,47.1267,26.600,86,0,1.54,AZ,1610243879


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [34]:
hotel_df = pd.DataFrame(ideal_df, columns=["City", "Country", "Lat", "Lng"])
hotel_df["Hotel Name"] = " "
hotel_df

,City,Country,Lat,Lng,Hotel Name
168,Vestmannaeyjar,IS,63.4427,-20.2734,
306,Hofn,IS,64.2539,-15.2082,
414,Taku,CN,39.0211,117.6469,
428,Ziarat,PK,30.3823,67.7282,
539,Bǝrdǝ,AZ,40.3744,47.1267,


In [45]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

hotel_name = []

radius = 5000
param = {
    "radius": radius,
    "type": "lodging",
    "keyword": "hotel",
    "key":g_key}

for index, row in hotel_df.iterrows():
    h_lat = row["Lat"]
    h_lng = row["Lng"]
    param['location'] = f"{h_lat}, {h_lng}"
    hotel_response = requests.get(url, params=param).json()
    
    try:
        hotel_name.append(hotel_response['results'][0]['name'])
    except:
        continue
hotel_name

['Puffin Nest Capsule Hostel',
 'Aurora Cabins Höfn Iceland',
 'InterContinental TIANJIN YUJIAPU HOTEL & RESIDENCES',
 'Grand Hotel',
 'Hotel']

In [47]:
hotel_df['Hotel Name'] = hotel_name
hotel_df

,City,Country,Lat,Lng,Hotel Name
168,Vestmannaeyjar,IS,63.4427,-20.2734,Puffin Nest Capsule Hostel
306,Hofn,IS,64.2539,-15.2082,Aurora Cabins Höfn Iceland
414,Taku,CN,39.0211,117.6469,InterContinental TIANJIN YUJIAPU HOTEL & RESID...
428,Ziarat,PK,30.3823,67.7282,Grand Hotel
539,Bǝrdǝ,AZ,40.3744,47.1267,Hotel


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
